In [1]:
# self-generated similar design?
# Simplify: each session one list?
import pandas as pd
import numpy as np
import pickle

In [2]:
# 1638 words
with open("../Data/wordpools/wasnorm_wordpool.txt") as f:
    items = f.readlines()
items = [i.replace("\n","") for i in items]
items = np.array(items)

In [3]:
nsubj = 100
nlist = 3
simu_sess_num = nsubj * nlist
n = 25
v = 5
n_exitem = 15
n_expair = 15
extra = 2 * (n_exitem * 1 + n_expair * 2)
wordpool = np.arange(1,1639)

In [4]:
df_study = pd.DataFrame()
df_test = pd.DataFrame()
# rng = np.random.default_rng(seed=42)
rng = np.random.default_rng()


subjidx = 1
subjsess = 0

for sess in range(simu_sess_num):
    tmp_df_study = pd.DataFrame()
    tmp_df_test1 = pd.DataFrame()
    tmp_df_test2 = pd.DataFrame()
    all_words = rng.choice(wordpool, 2*(n+v) + extra, replace=False)

    # study
    pres_words = all_words[0:2*n]
    pres_words = np.reshape(pres_words, (n,2))
    tmp_df_study['study_itemno1'] = pres_words[:,0]
    tmp_df_study['study_itemno2'] = pres_words[:,1]
    tmp_df_study['study_item1'] = items[pres_words[:,0]-1]
    tmp_df_study['study_item2'] = items[pres_words[:,1]-1]
    tmp_df_study['session'] = sess
    tmp_df_study['pair_idx'] = np.arange(n)
    tmp_df_study['subject'] = subjidx
    # df_study = pd.concat([df_study,tmp_df_study])

    # repeat 6 times, no continuous
    idx_order = []
    repeat_cnt = np.zeros(24)
    number_left = np.arange(24)
    last_pick = 0
    for i in range(24*6):
        pick = rng.choice(number_left)

        loop_cnt = 0
        while pick == last_pick:
            pick = rng.choice(number_left)
            loop_cnt += 1
            if loop_cnt > 100:
                print("fail")
                break

        idx_order.append(pick)
        repeat_cnt[pick] += 1
        if repeat_cnt[pick] == 6:
            np.delete(number_left, pick)
        last_pick = pick
    tmp_df_study = tmp_df_study.iloc[idx_order]
    df_study = pd.concat([df_study,tmp_df_study])

    # test conditions
    valid_words = all_words[0:2*(n+v)]
    valid_words = np.reshape(valid_words, (n+v,2))
    conditions = ["Same_Item"] * 6 + ["Different_Item"] * 6 + ["Intact_Pair"] * 3 + ["Item_Pair"] * 9 + ["Pair_Item"] * 6
    conditions = rng.permutation(conditions)

    test1_probe = []
    test2_probe = []
    for i, c in enumerate(conditions):
        order = rng.choice([0,1])
        this_pair = valid_words[i].tolist()
        if c == "Same_Item":
            test1_probe.append([this_pair[order],-1])
            test2_probe.append([this_pair[order],-1])
        elif c == "Different_Item":
            test1_probe.append([this_pair[order],-1])
            test2_probe.append([this_pair[1-order],-1])
        elif c == "Intact_Pair":
            test1_probe.append(this_pair)
            test2_probe.append(this_pair)
        elif c == "Item_Pair":
            test1_probe.append([this_pair[order],-1])
            test2_probe.append(this_pair)
        elif c == "Pair_Item":
            test1_probe.append(this_pair)
            test2_probe.append([this_pair[order],-1])
    test1_probe = np.array(test1_probe)
    test2_probe = np.array(test2_probe)

    # test1
    extra_words = all_words[2*(n+v):2*(n+v)+int(extra/2)]
    extra_items = extra_words[0:n_exitem]
    extra_pairs = np.reshape(extra_words[n_exitem:], (n_expair,2))
    tmp_df_test1['test_itemno1'] = np.concatenate((test1_probe[:,0], extra_items, extra_pairs[:,0]))
    tmp_df_test1['test_itemno2'] = np.concatenate((test1_probe[:,1], [-1]*n_exitem, extra_pairs[:,1]))
    tmp_df_test1['test_item1'] = items[tmp_df_test1['test_itemno1']-1]
    tmp_df_test1['test_item2'] = items[tmp_df_test1['test_itemno2']-1]
    tmp_df_test1['correct_ans'] = [1] * n + [0] * (v + n_exitem + n_expair)
    tmp_df_test1['pair_idx'] = np.concatenate((np.arange(n+v),[-1] * (n_exitem + n_expair)))
    tmp_df_test1['type'] =  np.concatenate((conditions, ["extra"] * (n_exitem + n_expair)))
    tmp_df_test1['test'] = 1
    tmp_df_test1['session'] = sess
    tmp_df_test1['subject'] = subjidx
    tmp_df_test1 = tmp_df_test1.iloc[rng.permutation(tmp_df_test1.index)].reset_index(drop=True)

    # test2
    extra_words = all_words[-int(extra/2):]
    extra_items = extra_words[0:n_exitem]
    extra_pairs = np.reshape(extra_words[n_exitem:], (n_expair,2))
    tmp_df_test2['test_itemno1'] = np.concatenate((test2_probe[:,0], extra_items, extra_pairs[:,0]))
    tmp_df_test2['test_itemno2'] = np.concatenate((test2_probe[:,1], [-1]*n_exitem, extra_pairs[:,1]))
    tmp_df_test2['test_item1'] = items[tmp_df_test2['test_itemno1']-1]
    tmp_df_test2['test_item2'] = items[tmp_df_test2['test_itemno2']-1]
    tmp_df_test2['correct_ans'] = [1] * n + [0] * (v + n_exitem + n_expair)
    tmp_df_test2['pair_idx'] = np.concatenate((np.arange(n+v),[-1] * (n_exitem + n_expair)))
    tmp_df_test2['type'] = np.concatenate((conditions, ["extra"] * (n_exitem + n_expair)))
    tmp_df_test2['test'] = 2
    tmp_df_test2['session'] = sess
    tmp_df_test2['subject'] = subjidx
    tmp_df_test2 = tmp_df_test2.iloc[rng.permutation(tmp_df_test2.index)].reset_index(drop=True)

    df_test = pd.concat([df_test,tmp_df_test1,tmp_df_test2])

    # update subject
    subjsess += 1
    if subjsess % nlist == 0:
        subjidx += 1
        subjsess = 0

df_study.reset_index(inplace = True, drop=True)
df_test.reset_index(inplace = True, drop=True)  

In [5]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,session,pair_idx,subject
0,1127,961,PRISON,NUCLEUS,0,23,1
1,1511,543,TROUT,EXPERT,0,15,1
2,922,634,MOTOR,GHOST,0,13,1
3,1127,961,PRISON,NUCLEUS,0,23,1
4,1634,1149,YOUTH,QUEEN,0,18,1
...,...,...,...,...,...,...,...
43195,12,1625,AIRPLANE,WRENCH,299,16,100
43196,1199,548,ROBIN,FACTORY,299,7,100
43197,164,359,BRACES,COMPUTER,299,8,100
43198,1568,1578,VINE,WAGON,299,18,100


In [6]:
df_test

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,pair_idx,type,test,session,subject
0,1634,1149,YOUTH,QUEEN,1,18,Intact_Pair,1,0,1
1,472,1004,DIRT,PAINTER,0,-1,extra,1,0,1
2,797,835,LAMB,LOAF,0,29,Intact_Pair,1,0,1
3,786,-1,KNIGHT,ZOO,0,-1,extra,1,0,1
4,522,-1,ELECTRON,ZOO,0,-1,extra,1,0,1
...,...,...,...,...,...,...,...,...,...,...
35995,1390,-1,STRAW,ZOO,0,-1,extra,2,299,100
35996,816,-1,LIBRARY,ZOO,1,5,Pair_Item,2,299,100
35997,1095,-1,POET,ZOO,0,-1,extra,2,299,100
35998,361,-1,CONCRETE,ZOO,0,27,Same_Item,2,299,100


In [7]:
# save
with open('../Data/simuS2_design.pkl', 'wb') as outp:
    pickle.dump(df_study, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(df_test, outp, pickle.HIGHEST_PROTOCOL)